In [3]:
# housekeeping: extract list of 2024B targets from xml file, since no easy list was left

import xml.etree.ElementTree as ET

tree = ET.parse("../../Proposals/Gemini/Gemini 2024B/Gemini_QEB.speckle_iq70.xml")
root = tree.getroot()
targets = set()
for target in root.findall("targets/sidereal"):
    name = target.find("name").text
    targets.add(name)

with open("../../Proposals/Gemini/Gemini 2024B/Gemini 2024B targets.csv", "w") as f:
    f.write("Gemini 2024B Proposal\n")
    f.write("All targets included in Gemini 2024B Proposal\n")
    for target in targets:
        f.write(f"{target}\n")

len(targets)

136

In [35]:
# use the target list creator to make a file that can be imported into the PIT
# we decided to use the same target list as the 2024B proposal

from functools import partial
from sqlite3 import connect
import sys

from astropy.table import Table

sys.path.append("../../")
import astropaul.targetlistcreator as tlc
import astropaul.gemini as gemini

conn = connect("../../django/TargetDB/db.sqlite3")

creator = tlc.TargetListCreator(connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="tess"),
    tlc.add_lists,
    gemini.add_gemini_speckle_params,
]

targets = Table.from_pandas(creator.calculate())
gemini_targets = targets[targets["list_Gemini_2024B_Proposal"]]
gemini_targets.rename_column("target_name", "Name")
gemini_targets.rename_column("ra", "RAJ2000")
gemini_targets.rename_column("dec", "DecJ2000")
gemini_targets.rename_column("pmra", "pmRA")
gemini_targets.rename_column("pmdec", "pmDec")
gemini_targets.rename_column("Vmag", "V")
gemini_targets.rename_column("gemini_iq70_speckle_program_time", "IQ70 Program Time")
gemini_targets.rename_column("gemini_iq85_speckle_program_time", "IQ85 Program Time")

gemini_targets[["Name", "RAJ2000", "DecJ2000", "pmRA", "pmDec", "V"]].write("Gemini 2025A Proposal.csv", overwrite=True)

# # break list into north & south and write to file
# cutpoint = 25
# lists = {
#     "North": gemini_targets[gemini_targets["DecJ2000"] > -cutpoint],
#     "South": gemini_targets[gemini_targets["DecJ2000"] < cutpoint],
# }
# for name, list in lists.items():
#     list[["Name", "RAJ2000", "DecJ2000", "pmRA", "pmDec", "V"]].write(f"Gemini 2025A Proposal {name}.csv", overwrite=True)

gemini_targets


Name,target_source,target_type,RAJ2000,DecJ2000,pmRA,pmDec,parallax,V,Teff,list_Featured_targets,list_Gemini_2024B_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_Speckle_Resolved,gemini_iq70_speckle_sequences,gemini_iq85_speckle_sequences,gemini_iq70_speckle_exposure,gemini_iq85_speckle_exposure,IQ70 Speckle Program Time,IQ85 Speckle Program Time
str14,str28,str6,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,int64,int64,float64,float64,float64,float64
TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.9253097979945,29.1345840372292,1.09215,-1.99413,0.347484,15.116,6445.0,False,True,False,False,True,False,False,False,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 58539786,Kostov 2023 arXiv:2309.14200,QuadEB,65.8340211217007,29.1427494807598,5.51137,-5.3655,0.83413,15.266,5597.0,False,True,False,False,True,False,False,False,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 219006972,Kostov 2023 arXiv:2309.14200,QuadEB,221.202871081398,66.3786726710139,6.64058,-3.44764,0.703547,14.312,5878.0,False,True,False,False,True,False,False,False,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 240256832,Kostov 2023 arXiv:2309.14200,QuadEB,322.296886956971,44.0333268274813,-16.4191,-17.5365,--,13.242,--,False,True,False,False,True,False,False,False,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 120911334,Kostov 2023 arXiv:2309.14200,QuadEB,73.8621033608711,38.3129564434802,-4.61153,0.315877,--,15.2,--,False,True,False,False,True,False,False,False,11,22,704.0,1408.0,1190.414688,2025.127776
TIC 430745185,Kostov 2023 arXiv:2309.14200,QuadEB,333.69028679781,49.2559110614137,-8.45255,-3.75994,0.561646,13.204,6282.0,False,True,False,False,True,False,False,False,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 371583423,Kostov 2023 arXiv:2309.14200,QuadEB,343.115417912764,55.4330449713393,-6.12292,-8.73148,--,13.881,--,False,True,False,False,True,False,False,False,7,14,448.0,896.0,886.882656,1418.0637120000001
TIC 414026507,Kostov 2022 arXiv:2202.05790,QuadEB,336.837716720964,56.7403620051295,-3.66449,-2.95647,0.242427,10.899,9187.0,False,True,True,True,False,True,True,False,5,10,320.0,640.0,735.1166400000001,1114.53168
TIC 121511673,Kostov 2023 arXiv:2309.14200,QuadEB,74.9719483022246,40.4020715331373,0.987281,-2.48414,0.306497,14.208,9336.0,False,True,False,False,True,False,False,False,11,22,704.0,1408.0,1190.414688,2025.127776


* Import the target list created above into the PIT
* In targets tab, sort by Dec
* Select & Copy all targets -90 < Dec < 25
* Switch to Observations tab and Paste
* Specify weather conditions and Zorro speckle (South)
* Select & Copy all targets -25 < Dec < 90
* Switch to Observations tab and Paste
* Specify weather conditions and `Alopeke speckle (North)
* Save PIT, which creates the .xml file

In [36]:
import xml.etree.ElementTree as ET

filename = "Gemini 2025A Proposal"

# first, get observation times for all targets
target_times = {name: (iq70, iq85) for name, iq70, iq85 in gemini_targets[["Name", "IQ70 Program Time", "IQ85 Program Time"]]},

# next, get the mapping of the nickname used in xml file to the TIC name
tree = ET.parse(f"{filename}.xml")
root = tree.getroot()
target_map = {}
for target in root.findall("targets/sidereal"):
    designator = target.attrib["id"]
    name = target.find("name").text
    target_map[designator] = name

#finally, walk through each observation entry and add observing time to it for iq70 conditions
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = target_times[target_name]
    progTime = observation.find("progTime")
    progTime.text = str(target_time[0])
    time = observation.find("time")
    time.text = str(target_time[0])
tree.write(f"{filename}_iq70.xml")

#repeat for iq85
for observation in root.findall("observations/observation"):
    target_name = target_map[observation.attrib["target"]]
    target_time = target_times[target_name]
    progTime = observation.find("progTime")
    progTime.text = str(target_time[1])
    time = observation.find("time")
    time.text = str(target_time[1])
tree.write(f"{filename}_iq85.xml")


KeyError: 'IQ70 Program Time'